<a href="https://colab.research.google.com/github/hsaurs/Projects/blob/main/AI_06_%EC%9C%A0%ED%98%95%EC%84%9D_Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install category_encoders

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data
names = ['age', 'workclass', 'fnlwgt', 'education', 'education num', 
                 'marital status', 'occupation', 'relationship', 'race', 'sex', 
                 'capital gain', 'capital loss', 'hours per week', 
                 'native country', 'income']
df = pd.read_csv('adult.data', header=None, names=names,na_values=['?', ' ?'])

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df = df.dropna()
df.reset_index(drop=True,inplace=True)
df.isna().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df = df.drop_duplicates()
df.duplicated().sum()

In [ ]:
df.shape

In [ ]:
df.head(4)

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
columns = ['workclass', 'education', 'marital status', 
           'occupation', 'relationship', 'race', 'sex',
           'native country']

In [ ]:
for i in columns:
    print(i,":",df[i].nunique(), np.unique(df[i]))

In [ ]:
for i in columns:
    df[i] =  df[i].str.replace(pat=r'[^\w]', repl=r'', regex=True)

In [ ]:
for i in columns:
    print(i,":",df[i].nunique(), np.unique(df[i]))

In [ ]:
df['Income'] = df['income'] == ' >50K'
df['Income'].nunique()

In [ ]:
df['Income'].value_counts()

In [ ]:
df['Income'] = df['Income']*1

In [ ]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(df,test_size=.2, random_state=2)
train.shape, val.shape

In [ ]:
from category_encoders import OrdinalEncoder
from sklearn.pipeline import make_pipeline

target = 'Income'
features = df.columns.drop([target,'income'])
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]

In [ ]:
## 1. DecisionTreeClassifier Model

from sklearn.tree import DecisionTreeClassifier

pipe1 = make_pipeline(
    OrdinalEncoder(),
    DecisionTreeClassifier(max_depth=5,random_state=2)
)

pipe1.fit(X_train, y_train)
print('검증 정확도: ', pipe1.score(X_val,y_val))

In [ ]:
## 2. RandomForestClassifier Model

from sklearn.ensemble import RandomForestClassifier

pipe2 = make_pipeline(
    OrdinalEncoder(),
    RandomForestClassifier(random_state=2)
)

pipe2.fit(X_train, y_train)
print('검증 정확도: ', pipe2.score(X_val,y_val))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

dists = {
    'randomforestclassifier__n_estimators': randint(50, 500),
    'randomforestclassifier__max_depth': [5, 10, 15, 20, None], 
    'randomforestclassifier__max_features': uniform(0, 1)
}

clf = RandomizedSearchCV(
    pipe2,
    param_distributions=dists,
    n_iter=50,
    cv=3,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)

clf.fit(X_train,y_train);
print('최적 하이퍼파라미터: ', clf.best_params_)

In [ ]:
pipe2 = clf.best_estimator_
pipe2.fit(X_train,y_train)
print('검증 정확도: ', pipe2.score(X_val,y_val))

In [ ]:
pd.DataFrame(clf.cv_results_).sort_values(by='rank_test_score').T

In [ ]:
from xgboost import XGBClassifier

pipe3 = make_pipeline(
    OrdinalEncoder(),
    XGBClassifier(random_state=2)
)

pipe3.fit(X_train, y_train)
print('검증 정확도: ', pipe3.score(X_val,y_val))

In [ ]:
dists = {
    'xgbclassifier__n_estimators': randint(50,500),
    'xgbclassifier__colsample_bytree': uniform(0.5,1),
    'xgbclassifier__subsample': uniform(0.6,1),
    'xgbclassifier__learning_rate': uniform(0,1),
    'xgbclassifier__max_depth': [5, 10, 15, 20, None], 
}

clf = RandomizedSearchCV(
    pipe3,
    param_distributions=dists,
    n_iter=50,
    cv=3,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)

clf.fit(X_train,y_train);

In [ ]:
pipe3 = clf.best_estimator_
pipe3.fit(X_train,y_train)
print('검증 정확도: ', pipe3.score(X_val,y_val))